In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
import re

from selenium.common.exceptions import WebDriverException


chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--blink-settings=imagesEnabled=false')


#mac
# driver = webdriver.Chrome(service = ChromeService(executable_path = './chromedriver-mac-arm64/chromedriver'),options = chrome_options)

# win
driver = webdriver.Chrome(service = ChromeService(executable_path = './chromedriver-mac-arm64/chromedriver'),options = chrome_options)

In [2]:
url_link = f'https://www.amazon.com'
driver.get(url_link)

In [3]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

while(True):
    try: 
        search = WebDriverWait(driver,0).until(EC.presence_of_element_located((By.ID, "twotabsearchtextbox")))
    except TimeoutException:
        try:
            search = WebDriverWait(driver,0).until(EC.presence_of_element_located((By.ID, "nav-bb-search")))
        except TimeoutException:       
            driver.refresh()
            continue
        else:
            break
    else:
        break

In [4]:
from selenium.webdriver.common.keys import Keys       

# category = input('enter the category of choice') 
# n_products = input('enter the number of products to be crawled')
n_products = 100
category = "camera"
search.send_keys(category,Keys.ENTER)

In [5]:
def total_pages():
    while(True):
        try: 
            total_pages = WebDriverWait(driver,0).until(EC.presence_of_element_located((By.XPATH, "//*[@class='s-pagination-item s-pagination-disabled']"))).text
        except TimeoutException: 
            driver.refresh()
            continue
        else:
            break
    return int(total_pages)

def products_in_page():    
    while(True):
        try: 
            elements_on_page = WebDriverWait(driver,3).until(EC.presence_of_all_elements_located((By.XPATH,'//*[contains(@data-cel-widget,"search_result_") and contains(@data-asin,"B0")]')))
        except TimeoutException: 
            driver.refresh()
            continue
        else:
            break      
    return elements_on_page

In [6]:
def links_in_page():
    while(True):
        try: 
            links = WebDriverWait(driver,0).until(EC.presence_of_all_elements_located((By.XPATH, "//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")))
        except TimeoutException: 
            driver.refresh()
            continue
        else:
            break
    return links

In [7]:
def unicode_remover(list_text):
    for t in range(len(list_text)):
        list_text[t] = re.sub(r'\\u[0-9a-fA-F]{4}', '', list_text[t])        
        list_text[t] = list_text[t].replace('\n', " ")
        
    return "".join(list_text)

In [8]:
products = []
links_set = set()
neighbor_links = set()
total_pages = total_pages()

In [9]:
def extract_product_name():
    try: 
        name = WebDriverWait(driver,0).until(EC.presence_of_all_elements_located((By.ID, "productTitle")))
        return [name[0].text]
        
    except Exception: 
        return ["None"]


In [10]:
def extract_product_price():
        try: 
            whole_price = WebDriverWait(driver,0).until(EC.presence_of_all_elements_located((By.XPATH, "//span[@class='a-price-whole']")))
            fraction_price = WebDriverWait(driver,0).until(EC.presence_of_all_elements_located((By.XPATH, "//span[@class='a-price-fraction']")))
            if whole_price != [] and fraction_price != []:
                return '.'.join([whole_price[0].text, fraction_price[0].text])
        except Exception: 
            return "None"
        

In [11]:
def extract_product_details():
        try: 
            details = WebDriverWait(driver,0).until(EC.presence_of_all_elements_located((By.XPATH, "//ul[@class='a-unordered-list a-vertical a-spacing-mini']//span[@class='a-list-item']")))
            if details != []:
                for i in range(len(details)):
                    details[i] = details[i].text
            return details
        except Exception: 
            return ["None"]

In [12]:
def extract_product_ratings():
        try: 
            rate = WebDriverWait(driver,0).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@id='averageCustomerReviews_feature_div']//span[@class = 'a-size-base a-color-base']")))
            if rate != []:
                 return rate[0].text
        except Exception: 
            return "None"

In [13]:
def extract_product_rates():
        try: 
            number_of_rates = WebDriverWait(driver,0).until(EC.presence_of_all_elements_located((By.ID, "acrCustomerReviewText")))
            if number_of_rates != []:
                return number_of_rates[0].text.split()[0]
        except Exception: 
            return "None"


In [14]:
def extract_product_reviews():
        try: 
            reviews = WebDriverWait(driver,0).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='a-expander-collapsed-height a-row a-expander-container a-expander-partial-collapse-container']")))
            if reviews != []:
                for i in range(len(reviews)):
                    reviews[i] = reviews[i].text
            return reviews
        except Exception: 
            return ["None"]

In [15]:
def extract_product_imageAddress():
    while(True):
        try: 
            Image_address  = WebDriverWait(driver,0).until(EC.presence_of_all_elements_located((By.ID, "landingImage")))
            if Image_address != []:
                return Image_address[0].get_attribute('src')
            
        except Exception: 
            return "None"

In [16]:
def extract_product_neighborsInfo():
    link = []
    try: 
        neighbors  = WebDriverWait(driver,0).until(EC.presence_of_all_elements_located((By.XPATH, "//*[contains(@id,'sp_detail2_') or contains(@id, 'CardInstanceLUGa0D_')]//a[@class='a-link-normal']")))
        if neighbors != []:
            for n in neighbors:
                if n not in links_set and n not in neighbor_links:
                    link.append(n.get_attribute('href'))
        return link[0:2]
    except Exception: 
        return []

In [17]:
def explore_neighbors(product_url):
    global driver
    
    while True:
        try:
            driver.get(product_url)
        except WebDriverException:
            driver = webdriver.Chrome(service = ChromeService(executable_path = './chromedriver-mac-arm64/chromedriver'),options = chrome_options)
            print("new driver created")
            while True:
                try:
                    WebDriverWait(driver,0).until(EC.presence_of_element_located((By.ID, "twotabsearchtextbox")))
                except TimeoutException:
                    try:
                        WebDriverWait(driver,0).until(EC.presence_of_element_located((By.ID, "nav-bb-search")))
                    except TimeoutException:       
                        driver.refresh()
                        continue
                    else:
                        break
        else:
            break
        
    name = extract_product_name()
    price = extract_product_price()
    details = extract_product_details()
    rate = extract_product_ratings()
    number_of_rating = extract_product_rates()
    reviews = extract_product_reviews()
    image = extract_product_imageAddress()
    
    
    product_info = {
        'Name': unicode_remover(name),
        'Price':price,
        'Details':unicode_remover(details),
        'Ratings':rate,
        'Rates':number_of_rating,
        'Reviews':unicode_remover(reviews),
        'Image':image,
        'Link':product_url
    }
    products.append(product_info)


In [18]:
def extract_product_info(product_url):
    global driver
    
    while True:
        try:
            driver.get(product_url)
        except WebDriverException:
            driver = webdriver.Chrome(service = ChromeService(executable_path = './chromedriver-mac-arm64/chromedriver'),options = chrome_options)
            print("new driver created")
            while True:
                try:
                    WebDriverWait(driver,0).until(EC.presence_of_element_located((By.ID, "twotabsearchtextbox")))
                except TimeoutException:
                    try:
                        WebDriverWait(driver,0).until(EC.presence_of_element_located((By.ID, "nav-bb-search")))
                    except TimeoutException:       
                        driver.refresh()
                        continue
                    else:
                        break
        else:
            break
    
    name = extract_product_name()
    price = extract_product_price()
    details = extract_product_details()
    rate = extract_product_ratings()
    number_of_rating = extract_product_rates()
    reviews = extract_product_reviews()
    image = extract_product_imageAddress()
    neighbors = extract_product_neighborsInfo()
    

    for i in neighbors:
        if not i in  links_set:
            neighbor_links.add(i)

    product_info = {
        'Name': unicode_remover(name),
        'Price':price,
        'Details':unicode_remover(details),
        'Ratings':rate,
        'Rates':number_of_rating,
        'Reviews':unicode_remover(reviews),
        'Image':image,
        'Link':product_url
    }
    
    products.append(product_info)


In [19]:
with open('amazon.html', 'w') as file:
    file.write(driver.page_source)
    file.close()

In [20]:
def extract_links():
    num_of_product = 0
    
    for i in range(1, total_pages + 1):
        driver.get(url_link+'/s?k='+category+'&page='+str(i))
        links = links_in_page()
        print(len(links),i)
        for link in range(len(links)):
                if (int(num_of_product) < int(n_products)):
                    links_set.add(links[link].get_attribute('href'))
                    num_of_product += 1
                else:
                    return

In [21]:
extract_links()

22 1
23 2
27 3
23 4
22 5


In [22]:
print(len(links_set))
print(len(neighbor_links))

100
0


In [23]:
count = 0
for link in links_set:
    print(count)
    print(link)
    extract_product_info(link)
    count += 1


0
https://www.amazon.com/KODAK-WPZ2-Waterproof-Digital-Optical/dp/B07ZRZLHH4/ref=sr_1_15?keywords=camera&qid=1706170088&sr=8-15
1
https://www.amazon.com/Photography-NIKICAM-Vlogging-Batteries-W03-Black6/dp/B0BQQVR7KD/ref=sr_1_10?keywords=camera&qid=1706170088&sr=8-10
2
https://www.amazon.com/sspa/click?ie=UTF8&spc=MTo1MjQ5MTg3NTMzMTU1MDI0OjE3MDYxNzAwODg6c3Bfc2VhcmNoX3RoZW1hdGljOjMwMDA5NDE5NjA3OTUwMjo6NDo6&url=%2FDigital-Camera-1080P-Vlogging-Students%2Fdp%2FB0CCRYF53X%2Fref%3Dsxin_12_pa_sp_search_thematic_top-features-in-reviews-t1_0_sspa%3Fcontent-id%3Damzn1.sym.73b9505a-7192-45cd-9fc3-cb080b40fa2a%253Aamzn1.sym.73b9505a-7192-45cd-9fc3-cb080b40fa2a%26cv_ct_cx%3Dcamera%26keywords%3Dcamera%26pd_rd_i%3DB0CCRYF53X%26pd_rd_r%3Db534b20d-02b2-4ef3-951c-761ec6a613d7%26pd_rd_w%3DTfbAm%26pd_rd_wg%3Dhsme2%26pf_rd_p%3D73b9505a-7192-45cd-9fc3-cb080b40fa2a%26pf_rd_r%3DBS5B1G88ZCBZ4K7FNTDK%26qid%3D1706170088%26sbo%3DRZvfv%252F%252FHxDF%252BO5021pAnSA%253D%253D%26sr%3D1-4-6101e845-7cb3-46f5-9461-6449

In [24]:
for link in neighbor_links:
    print(count)
    explore_neighbors(link)
    count += 1

100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282


In [25]:
import json
with open('./json.json', 'w') as json_file:
    json.dump(products , json_file)

In [26]:
len(products)

283

In [27]:
len(neighbor_links)

183

In [28]:
len(links_set)

100